In [1]:
from pathlib import Path
from tempfile import TemporaryDirectory
import flopy

temp_dir = TemporaryDirectory()
workspace = temp_dir.name
name = "mf6_model1"      #模型名称
length_units = "meters"  # 长度单位
time_units = "day"  # 时间单位

In [2]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=workspace)
# 将模拟进行时间离散化。perioddata 由 [perlen, nstp, tsmult] 构成，其中：
# perlen 是一个应力周期的长度。
# nstp 是一个应力周期的时间步数，为整数。
# tsmult 是连续时间步数的乘数，一般为 1，用于计算一个应力周期下时间步长，其计算方式可表示为：
flopy.mf6.ModflowTdis(sim, nper=10, perioddata=[[365.0, 1, 1.0] for _ in range(10)])
# 设置求解器
flopy.mf6.ModflowIms(sim)

# 建立模型 在 MODFLOW 6 中，渗流模型为 GWF，溶质运移模型为 GWT
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
flopy.mf6.ModflowGwfdis(gwf, nlay=3, nrow=4, ncol=5)
flopy.mf6.ModflowGwfic(gwf)
flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True)
flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 1.0], [(2, 3, 4), 0.0]])
budget_file = f"{name}.bud"
head_file = f"{name}.hds"
flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=budget_file,
    head_filerecord=head_file,
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)
print("Done creating simulation.")

Done creating simulation.
